<a href="https://colab.research.google.com/github/Bekamtz/projects/blob/main/Airbnb_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries and Data 

In [ ]:

import numpy as np
from PIL import Image
from os import path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import re 
#wordcloud
from nltk.corpus import stopwords
import string
import nltk
import nltk
nltk.download('punkt')
import nltk
nltk.download('stopwords')
import nltk
nltk.download('wordnet')
from os import path
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

from wordcloud import WordCloud, STOPWORDS

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
#import drive 
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
#import file 
df = pd.read_csv('gdrive/My Drive//Data/airbnb_1000.csv')

# Web Scrape Data 

Since there was no data online for HCMC, I had to scrape the data manually from the web

In [ ]:
# Import libraries
import sys
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pickle
from random import seed,shuffle

In [ ]:
# List containing unscraped URLs
errors = []


# Randomize listing URLs list
seed(3101)
shuffle(listings)

# Take 1200 URLs only
# We target 1000 but take a bit more because there will be errors in some listings
listings = listings[:1200]

name_data = []

data = {'room_info': [],
        'price': [],
        'review': [],
        'features': [],
        'no_review': [],
        'location': []}

tic = time.time()

# Scraper
for i in listings[600:]: # change this line to reverse search for i in listings[600:]:`
    
    # Print progress every 10 listings scraped
    if len(name_data) % 10 == 0:

      toc = time.time()

      print("No. of Scraped Listings: " + str(len(name_data)) + ". Epoch time: " + str(round(toc - tic, 2)) + " seconds.")

      tic = time.time()

    room_url = 'https://www.airbnb.com' + i

    driver = webdriver.Chrome('chromedriver', options = options)

    try:
      driver.get(room_url)

      time.sleep(3)

      try:

        name = driver.find_elements_by_class_name('_18hrqvin')[0].text

        room_info = [char.text for char in driver.find_elements_by_class_name('_czm8crp')][1:5]

        price = driver.find_elements_by_class_name('_doc79r')[0].text

        review = driver.find_elements_by_class_name('_tghtxy2')[0].text

        features = [char.text for char in driver.find_elements_by_class_name('_1p3joamp')[:4]]

        no_review = driver.find_elements_by_class_name('_1plk0jz1')[0].text

        location = driver.find_elements_by_id('neighborhood')[0].text

      except:

        name = room_info = price = review = features = no_review = location = None

        print("Unexpected error:", sys.exc_info()[0])

        errors.append(i)

    except:
      print("Page crash at", i)

      errors.append(i)

    name_data.append(name) 
    
    data['room_info'].append(room_info)

    data['price'].append(price)

    data['review'].append(review)

    data['features'].append(features)

    data['no_review'].append(no_review)

    data['location'].append(location)
  
    # Save data to file every 100 listings scraped
    if (len(name_data) % 100 == 0) and (len(name_data) > 0):
      df = pd.DataFrame(index = name_data, data = data, columns = data.keys())
      df.to_csv('/content/drive/My Drive/airbnb_{}.csv'.format(len(name_data)))
      print("Data saved with {} listings".format(len(name_data)))

    # Stop after scrapping 500 listings
    if len(name_data) == 500:
      break

In [ ]:
# Combine data parts
data_p1 = pd.read_csv('/content/drive/My Drive/airbnb/airbnb_500.csv')
data_p2 = pd.read_csv('/content/drive/My Drive/airbnb/airbnb_1000.csv')
data = pd.concat([data_p1, data_p2])
data.columns = ['name'] + list(data_p1.columns[1:])
data

# Cleaning Codes  

In [ ]:
df.head(1)


,Unnamed: 0,room_info,price,review,features,no_review,location
0,❤️StunningView ⭐PrivateBalcony♥️FREE Saltwater...,"['4 guests', '2 bedrooms', '2 beds', '2 baths']",$60,5.0,"['Entire home', 'Sparkling clean', 'Hannah is ...",(34 reviews),The neighborhood\nHannah’s place is located in...


Problems:


1.   Null Data 
2.   Data was saved as the wrong data type 
3.   Room information needed to be seperated into differate columns
4.   District information needed to be extracted from location and was not written in Vietnamese and English characters. 
5.   Features needed to be seperated into a new columns 




In [ ]:
# Changing Data Types was easy 
price = df['price']
price= price.str.replace('$','')
price = price.astype(int)
df['NewPrice'] = price



In [ ]:
#Caputuring Information was hard 
num_reviews=df['no_review']
num_reviews = num_reviews.str.findall('[0-9]+') 
num_reviews=num_reviews.astype(str)
num_reviews=num_reviews.str.replace("'",'') # there is probably an easier way to do this 
num_reviews=num_reviews.str.replace("[",'')
num_reviews=num_reviews.str.replace("]",'')
#reassign and change data type 
df["num_Reviews"]=num_reviews
df["num_Reviews"]=df["num_Reviews"].astype(int)

In [ ]:
# This was one of the hardest parts I had to figure out where District information was most frequently given after. 
dn = dn.str.findall('(?<=located in).((\D+)?)(\d?)(?=,\sH)') # this was tough finds it after the words located in and before the comma space H for Ho chi minh 


NameError: ignored

In [ ]:
#Then I had to use value_counts to figure out all the different ways hosts wrote their district. 
def txt_prep(text):
    replacements = [
        ['-', ', '], ['/ ', ', '], ['/', ', '], ['\(', ', '], [' and', ', '], [' &', ', '], ['\)', ''],
        ['Quận , Quận , 4', 'District 4'],
        ['Bến Thành, Bến Thành|Bình Thạnh,|Ben Thanh,Ben Thanh,|Bến Thành|Đa Kao, Đa Kao, |Đa Kao|phường , phường , 6|Bến Nghé|Bình Thạnh, Bình Thạnh|Bến Nghé, Bến Nghé, |Quận , Quận , 1|Nguyễn Thái Bình, Nguyễn Thái Bình,|Phường Nguyễn Thái Bìn, Phường Nguyễn Thái Bìn,|phường , phường , 5 |Nguyễn Cư Trinh, Nguyễn Cư Trinh, |Phạm Ngũ Lão, Phạm Ngũ Lão,|District , District , 1 |Cô Giang, Cô Giang,Phường , Phường , 6 |Nguyen Thai Binh, Nguyen Thai Binh,|phường ,phường ,5 |Cô Giang,Cô Giang,|Phường ,Phường ,6|District ,District ,1|phường ,phường ,5|District 1 ', 'District 1'],
        ['Ho Chi Minh City, Ho Chi Minh City,|Hồ Chí Minh City,Hồ Chí Minh City,|nan |Ho Chi Minh, Ho Chi Minh, |nan|Unknown ', 'Unknown'],
        ['Quận , Quận , 3|District , District , 3', 'District 3'],
        ['Phú Nhuận, Phú Nhuận', 'Phú Nhuận'],
        ['Tân Bình District,Tân Bình District,', 'Tân Bình'],
        ['ĐQuận ,Quận ,2|Quận ,Quận ,2', 'District 2'],
        [',  ', ', '], [', ,', ', '], ['\xa0', ' '],[',\s+', ','],
    
    ]
    for i, j in replacements:
        text = re.sub(str(i), str(j), str(text))
    return text
    

In [ ]:
df['Clean_District_Names'] = df['Clean_District_Names'].apply(txt_prep)

In [ ]:
#clean out Emojis found this online 
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

df['name'].apply(deEmojify)

In [ ]:
#make a Word Cloud
#Analyzing what features costs more ?
featuresDF = df[['features','NewPrice']]
featuresDFTopper = featuresDF.sort_values('NewPrice',ascending=[0])
featuresDFtop=featuresDFTopper.head(100) 
allfeatures = ''
for index,row in featuresDFtop.iterrows():
    p = re.sub('[^a-zA-Z]+',' ', row['features'])
    allfeatures+=p

allfeatures_data=nltk.word_tokenize(allfeatures)
filtered_data=[word for word in allfeatures_data if word not in stopwords.words('english')] 
wnl = nltk.WordNetLemmatizer() 
allfeatures_data=[wnl.lemmatize(data) for data in filtered_data]
allfeatures_words=' '.join(allfeatures_data)

#makes wordcloud
wordcloud = WordCloud(background_color="rgba(255, 255, 255, 0)", mode="RGBA",width = 1000, height = 700, max_words=100,color_func=lambda*args,**kwargs:(211,90,97)).generate(allfeatures_words)
plt.figure(figsize=(10,10))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
#Self Check in 
self_check = df['name']
self_check.astype(str)
self_check=self_check.str.findall('Self check-in')
df['self_check']=self_check
df['self_check'] = df['self_check'].apply(lambda x: 'Self check-in' if 'Self Check-in' in x else 'N/A' ) # the easier way to find text in the series 


In [ ]:
data = pd.read_csv('gdrive/My Drive//Data/AirbnbFinalFINALDrop4.csv')

In [ ]:
data

,room_type,bed_info,guest_info,NewPrice,review,NewnumberReviews,features,Clean_District_Names,clean_house_type
0,1 bedroom,2,4,37,4.54,103,"['Entire home', 'Self check-in', 'Great locati...",Unknown,Entire home
1,Studio,2,5,22,4.70,96,"['Entire home', 'Self check-in', 'Great locati...",District 1,Entire home
2,1 bedroom,1,2,35,4.72,36,"['Entire home', 'Hot tub', 'Read more about th...",District 1,Entire home
3,Studio,2,6,40,4.80,126,"['Entire home', 'Self check-in', 'Sparkling cl...",District 1,Entire home
4,2 bedrooms,2,6,53,4.83,29,"['Entire home', 'Self check-in', 'Great locati...",District 1,Entire home
...,...,...,...,...,...,...,...,...,...
973,1 bedroom,1,2,14,4.87,150,"['Private room in house', 'Self check-in', 'Sp...",Unknown,Shared home
974,1 bedroom,1,2,19,4.81,103,"['Entire home', 'Self check-in', 'Sparkling cl...",Unknown,Entire home
975,1 bedroom,1,2,15,4.98,41,"['Private room in apartment', 'Self check-in',...",District 4,Shared home
976,1 bedroom,1,2,31,4.75,81,"['Room in serviced apartment', 'Read more abou...",District 4,Shared home


Questions


1.   What's the average price per night in HCMC?
2.   What type of properties make up the HCMC Airbnb Market?
3.   What district is the most popular?
4.   How can hosts maximize their profits?

